<a href="https://colab.research.google.com/github/duncansamuelgeorgefreeman/colab/blob/master/object_detection_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pathlib

**Get tensorflow/models or cd to parent directory of repository.**

In [2]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path("models").exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2590, done.
remote: Counting objects: 100% (2590/2590), done.
remote: Compressing objects: 100% (2257/2257), done.
remote: Total 2590 (delta 504), reused 1360 (delta 295), pack-reused 0
Receiving objects: 100% (2590/2590), 124.10 MiB | 30.08 MiB/s, done.
Resolving deltas: 100% (504/504), done.
Checking out files: 100% (2497/2497), done.


**Compile protobufs and install the object detection package**

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [4]:
%%bash
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1017525 sha256=782549099055a1ab05cc21212858fbe9708c531c1cb9e9fae9671749e81baa2a
  Stored in directory: /tmp/pip-ephem-wheel-cache-h6hsi15s/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

**Import the Object Detection module**

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Model Preparation

### Variables
Any model exported using the export_inference_graph.py tool can be loaded here simply by changing the path.

For this example we use an SSD with MobileNet model. See detection model zooo for list of models that can be run out-of-the-box.

### Loader

In [0]:
def load_model(model_name):
  base_url = "http://download.tensorflow.org/models/object_detection/"
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
      fname = model_name,
      origin = base_url + model_file,
      untar = True
  )
  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

In [0]:
# patches

# patch tf1 into utils.ops
utils_ops.tf = tf.compat.v1

# patch location of gfile
tf.gfile = tf.io.gfile

# Loading Label Map

Maps inidces to category names, so for example when our convolution network predicts 5 we know that corresponds to airplane. This example uses internal functions, but anything that returns a dictionary mapping ints to the appropriate string label works fine.

In [0]:
# List of strings that is used to add the correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

**Test on 2 images**

**To test the code with other images, just add path to the images to the TEST_IMAGE_PATHS.**

In [14]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('models/research/object_detection/test_images/image1.jpg'),
 PosixPath('models/research/object_detection/test_images/image2.jpg')]

# Detection

**Load an object detection model**

In [15]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

76537856/76534733 [==============================] - 1s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
# Check model's input signature - it expects a batch of 3-color images of type uint8
print(detection_model.inputs)

AttributeError: ignored